# Points intersecction

In [5]:
import pandas as pd
import json
import requests
import numpy as np

## Predifined annual

**Group names**

|Group full 	               |Group short    |Column name          |
|------------------------------|---------------|---------------------|
|Overall Water Risk            |TOT            |w_awr_def_tot_label  |
|Water Quantity Risk           |QAN            |w_awr_def_qan_label  |
|Water Quality Risk            |QAL            |w_awr_def_qal_label  |
|Regulatory and Reputational   |RRR            |w_awr_def_rrr_label  |

**Indicator names**

|Indicator full 	                 |Indicator short    |Column name  |
|------------------------------------|-------------------|-------------|
|Baseline Water Stress               |bws                |bws_label    |    
|Baseline Water Depletion            |bwd                |bwd_label    |
|Groundwater Table Decline           |gtd                |gtd_label    |
|Interannual Variability             |iav                |iav_label    |
|Seasonal Variability                |sev                |sev_label    |
|Drought Risk                        |drr                |drr_label    |
|Riverine Flood Risk                 |rfr                |rfr_label    |
|Coastal Flood Risk                  |cfr                |cfr_label    |
|Untreated Collected Wastewater      |ucw                |ucw_label    |
|Coastal Eutrophication Potential    |cep                |cep_label    |
|Unimproved/no drinking water        |udw                |udw_label    |
|Unimproved/no sanitation            |usa                |usa_label    |
|RepRisk Index                       |rri                |rri_label    |


`Sql:`

```sql
with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id 
                FROM unnest(Array['POINT(-4.04296875 39.90973623453719)', 'POINT(10.01953125 51.39920565355378)', 'POINT(-6.50390625 32.84267363195431)', 'POINT(0.3515625 41.376808565702355)']) as points
               )

SELECT points_id::numeric, points.the_geom, r.string_id, w.name_0 as country, w.name_1 as province, b.maj_name as basin,
       w_awr_def_tot_label::text as overall_water_risk,             
       w_awr_def_qan_label::text as water_quantity_risk,            
       w_awr_def_qal_label::text as water_quality_risk,            
       w_awr_def_rrr_label::text as regulatory_and_reputational,    
       bws_label::text as baseline_water_stress,                       
       bwd_label::text as baseline_water_depletion,                
       gtd_label::text as groundwater_table_decline,               
       iav_label::text as interannual_variability,                 
       sev_label::text as seasonal_variability,                    
       drr_label::text as drought_risk,                            
       rfr_label::text as riverine_flood_risk,                     
       cfr_label::text as coastal_flood_risk,                      
       ucw_label::text as untreated_collected_wastewater,          
       cep_label::text as coastal_eutrophication_potential,        
       udw_label::text as nnimproved_drinking_water,            
       usa_label::text as unimproved_sanitation,                
       rri_label::text as repRisk_index                                                             
FROM water_risk_indicators_annual r
LEFT JOIN y2018m12d06_rh_master_shape_v01 g on r.aq30_id=g.aq30_id
RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom)
LEFT JOIN gadm_wri w on r.gid_1=w.gid_1 
LEFT JOIN basin_names b on r.pfaf_id=b.pfaf_id                                
```

**Create a function**

We create a function in carto to deal with all this operations.

```sql
DROP FUNCTION get_aqpoints_annual(points_array TEXT);

DROP TYPE point_intersection_annual;

CREATE TYPE point_intersection_annual AS (points_id numeric, the_geom geometry, string_id text, country text, region text, basin text, overall_water_risk text, water_quantity_risk text, water_quality_risk text, regulatory_and_reputational text, baseline_water_stress text, baseline_water_depletion text, groundwater_table_decline text, interannual_variability text, seasonal_variability text, drought_risk text, riverine_flood_risk text, coastal_flood_risk text, untreated_collected_wastewater text, coastal_eutrophication_potential text, unimproved_drinking_water text, unimproved_sanitation text, reprisk_index text);

CREATE OR REPLACE FUNCTION get_aqpoints_annual(points_array TEXT)
	RETURNS SETOF point_intersection_annual as $$
	BEGIN
	RETURN query EXECUTE  'with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points ) SELECT points_id::numeric, points.the_geom, r.string_id, w.name_0 as country, w.name_1 as province, b.maj_name as basin, w_awr_def_tot_label::text as overall_water_risk, w_awr_def_qan_label::text as water_quantity_risk, w_awr_def_qal_label::text as water_quality_risk, w_awr_def_rrr_label::text as regulatory_and_reputational, bws_label::text as baseline_water_stress, bwd_label::text as baseline_water_depletion, gtd_label::text as groundwater_table_decline, iav_label::text as interannual_variability, sev_label::text as seasonal_variability, drr_label::text as drought_risk, rfr_label::text as riverine_flood_risk, cfr_label::text as coastal_flood_risk, ucw_label::text as untreated_collected_wastewater, cep_label::text as coastal_eutrophication_potential, udw_label::text as nnimproved_drinking_water, usa_label::text as unimproved_sanitation, rri_label::text as repRisk_index FROM water_risk_indicators_annual r LEFT JOIN y2018m12d06_rh_master_shape_v01 g on r.aq30_id=g.aq30_id RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom) LEFT JOIN gadm_wri w on r.gid_1=w.gid_1 LEFT JOIN basin_names b on r.pfaf_id=b.pfaf_id';

	END

$$ language 'plpgsql';
```
**Final query**

`Sql Example:`
```sql
SELECT * FROM get_aqpoints_annual('[''POINT(-4.04296875 39.90973623453719)'', ''POINT(0.3515625 41.376808565702355)'', ''POINT(10.01953125 51.39920565355378)'', ''POINT(-6.50390625 32.84267363195431)'']')
```

`Sql Template:`
``` sql
SELECT * FROM get_aqpoints_annual({points_array})
```

## Monthly

`Sql:`

```sql
with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id 
                FROM unnest(Array['POINT(-4.04296875 39.90973623453719)', 'POINT(10.01953125 51.39920565355378)', 'POINT(-6.50390625 32.84267363195431)', 'POINT(0.3515625 41.376808565702355)']) as points
               )

SELECT pfaf_id::numeric, points.the_geom, 
       bws_label as baseline_water_stress,
       bwd_label as baseline_water_depletion,
       iav_label as interannual_variability
                                                            
FROM water_risk_indicators_monthly r
WHERE month = 1
LEFT JOIN y2018m12d06_rh_master_shape_v01 g on r.pfaf_id=g.pfaf_id
RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom)                                  
```

## Custom and Predifined  annual
`Sql:`
```sql
with r as (SELECT * FROM (
                          SELECT unnest(Array[4.0, 4.0, 2.0, 0.5, 4.0, 4.0, 1.0, 1.0, null, null, null, null, null]) as weights,
                                 unnest(Array['bws', 'bwd', 'iav', 'sev', 'gtd', 'drr', 'rfr', 'cfr', 'ucw', 'cep', 'udw', 'usa', 'rri']) as indicator
                         ) as a 
           WHERE weights is not null
          ),
     s as (SELECT indicator, weights/sum_weights as weight_fraction
           FROM (
                 SELECT sum(weights) over () as sum_weights,  weights, indicator
                 FROM r
                ) as a
          ),
     u as (SELECT aq30_id, sum(weight_fraction) as weight_fraction, sum(weighted_score) as weighted_score
           FROM (
                 SELECT t.group_short, t.indicator, t.score, t.aq30_id, s.weight_fraction, t.score*s.weight_fraction as weighted_score
                 FROM water_risk_indicators_normalized t
                 LEFT JOIN s on t.indicator=s.indicator
                ) as u
            GROUP BY aq30_id
          ),
     v as (SELECT aq30_id, weighted_score, weight_fraction, weighted_score/weight_fraction as raw
           FROM u
          ),
points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id 
           FROM unnest(Array['POINT(3.515625 47.040182144)', 'POINT(30.5859374 53.5403073)', 'POINT(43.59375 30.145127)', 'POINT(22.8515625 9.449061826)']) as points
          )
          
SELECT v.aq30_id, round(v.raw::numeric,6) as water_risk, points.the_geom
FROM v                                              
LEFT JOIN y2018m12d06_rh_master_shape_v01 g on v.aq30_id=g.aq30_id
RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom)                             
ORDER BY aq30_id
```    

**Create a function**

We create a function in carto to deal with all this operations.

```sql
DROP FUNCTION get_aqpoints_annual(weights_array TEXT, points_array TEXT);

DROP TYPE point_intersection_annual;

CREATE TYPE point_intersection_annual AS (aq30_id integer, water_risk numeric, the_geom geometry);

CREATE OR REPLACE FUNCTION get_aqpoints_annual(weights_array TEXT, points_array TEXT)
	RETURNS SETOF point_intersection_annual as $$
	BEGIN
	RETURN query EXECUTE  'with r as (SELECT * FROM (SELECT unnest(Array'|| weights_array ||') as weights, unnest(Array[''bws'', ''bwd'', ''iav'', ''sev'', ''gtd'', ''drr'', ''rfr'', ''cfr'', ''ucw'', ''cep'', ''udw'', ''usa'', ''rri'']) as indicator) as a WHERE weights is not null), s as (SELECT indicator, weights/sum_weights as weight_fraction FROM (SELECT sum(weights) over () as sum_weights,  weights, indicator FROM r) as a), u as (SELECT aq30_id, sum(weight_fraction) as weight_fraction, sum(weighted_score) as weighted_score FROM (SELECT t.group_short, t.indicator, t.score, t.aq30_id, s.weight_fraction, t.score*s.weight_fraction as weighted_score FROM water_risk_indicators_normalized t LEFT JOIN s on t.indicator=s.indicator) as u GROUP BY aq30_id), v as (SELECT aq30_id, weighted_score, weight_fraction, weighted_score/weight_fraction as raw FROM u), points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points) SELECT v.aq30_id, round(v.raw::numeric,6) as water_risk, points.the_geom FROM v LEFT JOIN y2018m12d06_rh_master_shape_v01 g on v.aq30_id=g.aq30_id RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom) ORDER BY aq30_id';

	END

$$ language 'plpgsql';
```
**Final query**

`Sql Example:`
```sql
SELECT * FROM get_aqpoints_annual('[4.0, 4.0, 2.0, 0.5, 4.0, 4.0, 1.0, 1.0, null, null, null, null, null]','[''POINT(3.515625 47.040182144)'', ''POINT(30.5859374 53.5403073)'', ''POINT(43.59375 30.145127)'', ''POINT(22.8515625 9.449061826)'']')
```

`Sql Template:`
``` sql
SELECT * FROM get_aqpoints_annual({weights_array},{points_array})
```

## Predifined monthly
`Sql:`
```sql
with r as (SELECT pfaf_id, bws_cat, bwd_cat, iav_cat  
           FROM water_risk_indicators_monthly WHERE month = 1
          ),
points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id 
           FROM unnest(Array['POINT(3.515625 47.040182144)', 'POINT(30.5859374 53.5403073)', 'POINT(43.59375 30.145127)', 'POINT(22.8515625 9.449061826)']) as points
          )

SELECT r.pfaf_id, r.bws_cat::numeric as baseline_water_stress, r.bwd_cat::numeric as baseline_water_depletion, r.iav_cat::numeric as interannual_variability, points.the_geom
FROM r
LEFT JOIN y2018m12d06_rh_master_shape_v01 g on r.pfaf_id=g.pfaf_id
RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom)  
ORDER BY r.pfaf_id                        
```

**Create a function**

We create a function in carto to deal with all this operations.

```sql
DROP FUNCTION get_aqpoints_monthly(month TEXT, points_array TEXT);

DROP TYPE point_intersection_monthly;

CREATE TYPE point_intersection_monthly AS (pfaf_id integer, baseline_water_stress numeric, baseline_water_depletion numeric, interannual_variability numeric, the_geom geometry);

CREATE OR REPLACE FUNCTION get_aqpoints_monthly(month TEXT, points_array TEXT)
	RETURNS SETOF point_intersection_monthly as $$
	BEGIN
	RETURN query EXECUTE  'with r as (SELECT pfaf_id, bws_cat, bwd_cat, iav_cat FROM water_risk_indicators_monthly WHERE month = '|| month ||'), points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points) SELECT r.pfaf_id, r.bws_cat::numeric as baseline_water_stress, r.bwd_cat::numeric as baseline_water_depletion, r.iav_cat::numeric as interannual_variability, points.the_geom FROM r LEFT JOIN y2018m12d06_rh_master_shape_v01 g on r.pfaf_id=g.pfaf_id RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom) ORDER BY r.pfaf_id';

	END

$$ language 'plpgsql';
```

`Sql Example:`
```sql
SELECT * FROM get_aqpoints_monthly('1','[''POINT(3.515625 47.040182144)'', ''POINT(30.5859374 53.5403073)'', ''POINT(43.59375 30.145127)'', ''POINT(22.8515625 9.449061826)'']')
```

`Sql Template:`
``` sql
SELECT * FROM  get_aqpoints_monthly({month},{points_array})
```

## Projected Changes
`Sql:`
```sql
with r as (SELECT basinid, type, model, indicator, label, value, year FROM water_risk_indicators 
           WHERE year = 2030 and type = 'absolute' and indicator = 'seasonal_variability' and model = 'pessimistic'
          ),
points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id 
           FROM unnest(Array['POINT(3.515625 47.040182144)', 'POINT(30.5859374 53.5403073)', 'POINT(43.59375 30.145127)', 'POINT(22.8515625 9.449061826)']) as points
          )

SELECT r.basinid, r.value, r.label, points.the_geom
FROM r
LEFT JOIN wri_subcatchements g on r.basinid=g.basinid
RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom)  
ORDER BY r.basinid
```

**Create a function**

We create a function in carto to deal with all this operations.

```sql
DROP FUNCTION get_aqpoints_projected(year TEXT, type TEXT, indicator TEXT, model TEXT, points_array TEXT);

DROP TYPE point_intersection_projected;

CREATE TYPE point_intersection_projected AS (basinid integer, value numeric, label text, the_geom geometry);

CREATE OR REPLACE FUNCTION get_aqpoints_projected(year TEXT, type TEXT, indicator TEXT, model TEXT, points_array TEXT)
	RETURNS SETOF point_intersection_projected as $$
	BEGIN
	RETURN query EXECUTE  'with r as (SELECT basinid, type, model, indicator, label, value, year FROM water_risk_indicators WHERE year = '|| year ||' and type = type and indicator = indicator and model = model), points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points) SELECT r.basinid, r.value, r.label, points.the_geom FROM r LEFT JOIN wri_subcatchements g on r.basinid=g.basinid RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom) ORDER BY r.basinid';

	END

$$ language 'plpgsql';
```

`Sql Example:`
```sql
SELECT * FROM get_aqpoints_projected('2030', ''absolute'', ''seasonal_variability'', ''pessimistic'', '[''POINT(3.515625 47.040182144)'', ''POINT(30.5859374 53.5403073)'', ''POINT(43.59375 30.145127)'', ''POINT(22.8515625 9.449061826)'']')
```

'with r as (SELECT basinid, type, model, indicator, label, value, year FROM water_risk_indicators WHERE year = '|| year ||' and type = '|| type ||' and indicator = '|| indicator ||' and model = '|| model ||'), points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points) SELECT r.basinid, r.value, r.label, points.the_geom FROM r LEFT JOIN wri_subcatchements g on r.basinid=g.basinid RIGHT JOIN points on  st_intersects(g.the_geom, points.the_geom) ORDER BY r.basinid'

# Points intersecction

```sql
with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id 
                FROM unnest(Array'|| points_array ||') as points), 
     ranges as (SELECT basinid, average, min(average) over (), max(average) over (), 
                       bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s 
                FROM (SELECT basinid, ('|| weights ||')/('|| weights_sum ||') as average, 
                             bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s 
                      FROM water_risk_weights) initial) 
                      
SELECT points_id::numeric, ranges.basinid::numeric, points.the_geom, (((average-min)/(max-min))*5)::numeric as water_risk, 
       bws_s::numeric as baseline_water_stress, wsv_s::numeric as interannual_variability, sv_s::numeric as seasonal_variability, 
       hfo_s::numeric as flood_occurrence, dro_s::numeric as drought_severity, stor_s::numeric as upstream_storage, 
       gw_s::numeric as groundwater_stress, wri_s::numeric as return_flow_ratio, eco_s_s::numeric as upstream_protected_land, 
       mc_s::numeric as media_coverage, wcg_s::numeric as access_to_water, eco_v_s::numeric as threatened_amphibians 
FROM ranges 
INNER JOIN wri_subcatchements on ranges.basinid=wri_subcatchements.basinid 
RIGHT JOIN points on  st_intersects(wri_subcatchements.the_geom, points.the_geom)
```

Procedure definition:


```sql

DROP FUNCTION get_aqpoints(weights_raw TEXT, points_array TEXT); 

DROP TYPE weight_intersection;

CREATE TYPE weight_intersection AS (points_id numeric, basinid numeric, the_geom geometry, water_risk numeric, water_stress numeric, interannual_variability numeric, seasonal_variability numeric, flood_occurrence numeric, drought_severity numeric, upstream_storage numeric, groundwater_stress numeric, return_flow_ratio numeric, upstream_protected_land numeric, media_coverage numeric, access_to_water numeric, threatened_amphibians numeric);

CREATE OR REPLACE FUNCTION get_aqpoints(weights_raw TEXT, points_array TEXT) 
	RETURNS SETOF weight_intersection as $$
	DECLARE
    query1 TEXT;
    weights TEXT;
    weights_sum TEXT;
	BEGIN
        query1:='select array_to_string(array_agg(col::text ||''*''|| weights::text), ''+ '') as weights, sum(weights)::text as weights_sum from (select unnest(Array'|| weights_raw||') as weights, unnest(array[''bws_s'', ''wsv_s'', ''sv_s'', ''hfo_s'', ''dro_s'', ''stor_s'', ''gw_s'', ''wri_s'', ''eco_s_s'', ''mc_s'', ''wcg_s'', ''eco_v_s'']) as col) as tables where weights is not null';
        
        EXECUTE query1 into weights, weights_sum;
        
		RETURN query EXECUTE 'with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points), ranges as (select basinid, average, min(average) over (), max(average) over (), bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s from (SELECT basinid, ('|| weights ||')/('|| weights_sum ||') as average, bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s FROM water_risk_weights) initial) select points_id::numeric, ranges.basinid::numeric, points.the_geom, (((average-min)/(max-min))*5)::numeric as water_risk, bws_s::numeric as baseline_water_stress, wsv_s::numeric as interannual_variability, sv_s::numeric as seasonal_variability, hfo_s::numeric as flood_occurrence, dro_s::numeric as drought_severity, stor_s::numeric as upstream_storage, gw_s::numeric as groundwater_stress, wri_s::numeric as return_flow_ratio, eco_s_s::numeric as upstream_protected_land, mc_s::numeric as media_coverage, wcg_s::numeric as access_to_water, eco_v_s::numeric as threatened_amphibians from ranges inner join wri_subcatchements on ranges.basinid=wri_subcatchements.basinid right join points on  st_intersects(wri_subcatchements.the_geom, points.the_geom)';
	
	END
$$ language 'plpgsql';

```



Sql Example:

```sql
Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]','[''POINT(84.8085584935 -14.20067639)'', ''POINT(54.0392656274 -70.8898132233)'', ''POINT(-28.5832686897 -7.71134965117)'', ''POINT(47.9458596199 82.5599787066)'', ''POINT(50.8126903314 -46.0154993389)'', ''POINT(-62.5229253542 -9.68983337791)'', ''POINT(-34.7977234627 84.9984574252)'', ''POINT(-80.1102876685 -33.9286081419)'', ''POINT(22.6686500117 -85.6713992254)'', ''POINT(44.359802466 -27.7294728889)'']')
```



Sql Template:

``` sql
Select * from get_aqpoints({weights_scheeme},{points array})

```





In [4]:
payload = {'q': "Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]','[''POINT(84.8085584935 -14.20067639)'', ''POINT(54.0392656274 -70.8898132233)'', ''POINT(-28.5832686897 -7.71134965117)'', ''POINT(47.9458596199 82.5599787066)'', ''POINT(50.8126903314 -46.0154993389)'', ''POINT(-62.5229253542 -9.68983337791)'', ''POINT(-34.7977234627 84.9984574252)'', ''POINT(-80.1102876685 -33.9286081419)'', ''POINT(22.6686500117 -85.6713992254)'', ''POINT(44.359802466 -27.7294728889)'']')"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(1)

,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0101000020E6100000139F1F6CBF3355403942480EBF66...,NaN,NaN,NaN,NaN,NaN


# Anexo: Points stress test

test points; 100; 500; 1000; 100000;

In [4]:
t = 180 * np.random.rand(1000000,2) - 90

'POINT(-72.0 42.2)'

In [5]:
example1=[]
for point in t:
    example1.append('\'\'POINT(' +str(point[0])+ ' ' + str(point[1]) +')\'\'')

In [6]:
t10 = '['+', '.join(example1[:10])+']'
t100 = '['+', '.join(example1[:100])+']'
t1000 = '['+', '.join(example1[:1000])+']'
t10000 = '['+', '.join(example1[:10000])+']'
print(t10)
print(len(t10))
print(len(t100))
print(len(t1000))
print(len(t10000))

[''POINT(43.9701276111 11.1301394129)'', ''POINT(86.0466406701 13.2688619157)'', ''POINT(-53.8696343831 64.6113269282)'', ''POINT(87.4130123196 -23.032929918)'', ''POINT(57.2656143129 -88.2644839221)'', ''POINT(74.5658970947 -7.62928716917)'', ''POINT(3.51858671798 -12.3788819738)'', ''POINT(-18.5802293632 41.4033717967)'', ''POINT(16.2846733241 55.3067910163)'', ''POINT(78.6515080867 -49.5037394613)'']
406
4075
40809
407990


In [8]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t10 +"\')"

r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(2)

,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0101000020E6100000804F3D242DFC4540FCA614A2A142...,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,0101000020E61000008E2F2629FC8255409024DE44A889...,NaN,NaN,NaN,NaN,NaN


In [63]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t100 +"\')"
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)

tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

<Response [200]>


,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress


In [5]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]','[''Point(72.421875, 56.559482)'', ''Point(18.6328125, 10.8333059)'', ''Point(-109.6875, 42.03297)'']')"
r = requests.post('https://wri-01.carto.com/api/v2/sql', data=payload)
if r.status_code != 200:
    issue = json.loads(r.text)
    print(issue)
else:
    tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
    tableStructure.head(10)
        



{'error': ['parse error - invalid geometry'], 'hint': '"Point(72.421875, " <-- parse error at position 17 within geometry'}
